<a href="https://colab.research.google.com/github/mounilgarg/dataMining/blob/main/Androsensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# displays complete androsensor app dataset 
import csv
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from collections import defaultdict

rows=[]
X=[]
Y=[]
Z=[]
proximity=[]
soundLevel=[]
latitude=[]
longitude=[]
altitude=[]
altGoogle=[]
speed=[]
accuracy=[]
orient=[]
satInRange=[]
time=[]
date=[]
with open("/content/drive/MyDrive/csvData/record3.csv",encoding = 'unicode_escape') as file:
    reader = csv.DictReader(file)
    
    for row in reader:
      X.append(row['ACCELEROMETER X (m/sÂ²)'])
      Y.append(row['ACCELEROMETER Y (m/sÂ²)'])
      Z.append(row['ACCELEROMETER Z (m/sÂ²)'])
      proximity.append(row['PROXIMITY (i)'])
      soundLevel.append(row['SOUND LEVEL (dB)'])
      latitude.append(row['LOCATION Latitude : '])
      longitude.append(row['LOCATION Longitude : '])
      altitude.append(row['LOCATION Altitude ( m)'])
      altGoogle.append(row['LOCATION Altitude-google ( m)'])
      speed.append(row['LOCATION Speed ( Kmh)'])
      accuracy.append(row['LOCATION Accuracy ( m)'])
      orient.append(row['LOCATION ORIENTATION (Â°)'])
      satInRange.append(row['Satellites in range'])
      time.append(row['Time since start in ms '])
      date.append(row['YYYY-MO-DD HH-MI-SS_SSS'])
    
    numRows=len(X)

    i=0
    while i<numRows:
      rowData=[X[i]]
      rowData.append(Y[i])
      rowData.append(Z[i])
      rowData.append(proximity[i])
      rowData.append(soundLevel[i])
      rowData.append(latitude[i])
      rowData.append(longitude[i])
      rowData.append(altitude[i])
      rowData.append(altGoogle[i])
      rowData.append(speed[i])
      rowData.append(accuracy[i])
      rowData.append(orient[i])
      rowData.append(satInRange[i])
      rowData.append(time[i])
      rowData.append(date[i])
      
      rows.append(rowData)
      i+=1
    
    df = pd.DataFrame(rows, columns = ['ACCELEROMETER X (m/s²)','ACCELEROMETER Y (m/s²)','ACCELEROMETER Z (m/s²)','PROXIMITY (i)'
    ,'SOUND LEVEL (dB)','LOCATION Latitude','LOCATION Longitude','LOCATION Altitude ( m)',
    'LOCATION Altitude-google ( m)','LOCATION Speed ( Kmh)','LOCATION Accuracy ( m)','LOCATION ORIENTATION (°)','Satellites in range','Time since start in ms ','YYYY-MO-DD HH-MI-SS_SSS'])

print(df)
# print(rows)

    ACCELEROMETER X (m/s²) ACCELEROMETER Y (m/s²) ACCELEROMETER Z (m/s²)  \
0                 -0.41659                3.96001                8.66701   
1                  1.97761               -0.35913                9.99819   
2                  0.50278               -0.15802               11.10431   
3                  0.06704               -0.02394                9.97424   
4                 -0.08619                0.13886               10.23282   
..                     ...                    ...                    ...   
357                2.00155                -0.1245               16.36676   
358               -2.27449               -2.72939                9.72046   
359                0.85234                1.38385                9.32302   
360               -0.17717                1.62806                8.63828   
361               -1.48919                0.97683                9.30866   

    PROXIMITY (i) SOUND LEVEL (dB) LOCATION Latitude LOCATION Longitude  \
0           

In [3]:
# Data preprocessing: windowing(time domain)
import numpy as np

# creates windows of specified length, and given stride
def windowing(X,Y,Z,outX,outY,outZ,w,s,numRows):
  i=0
  row=0

  # increments the stride to form total  number of windows
  while i<=numRows-1:
    
    j=i 
    col=0

    # this loop creates a window for current stride
    while col<w and j<=numRows-1:
      outX[row][col]=X[j]
      outY[row][col]=Y[j]
      outZ[row][col]=Z[j]
      j+=1
      col+=1
    i+=s
    row+=1
  out={0:outX, 1:outY, 2:outZ};
  return out  

#calculation of stats for each window
def statistics(out,numWind):
  numWind=(int(np.ceil(numWind)))
  c=0
  result={}
  # outer loop for each dimention: x,y,z
  i=0
  while(i<3):
    # inner loop for each window of current axis
    j=0
    while j<numWind:
      if j==0:
        m=[out[i][j].mean()]
        s=[out[i][j].std()]
      else:
        m.append(out[i][j].mean())
        s.append(out[i][j].std())
      j+=1
    result[c]=m
    result[c+1]=s
    c+=2
    i+=1
  return result              

# X,Y,Z resp contais acceleration in corr axis
X=np.array(X)
Y=np.array(Y)
Z=np.array(Z)
w=256 #window size
s=16 #stride

# numRows=total number of values in X,Y or Z
# numWind= number of windows created
numRows=len(X)
numWind=numRows/s

# rows=ceil(X.size/s) ; cols=w
# outX,outY,outZ contain all the windows for corr axis
outX=np.zeros((int(np.ceil(numWind)),w))
outY=np.zeros((int(np.ceil(numWind)),w))
outZ=np.zeros((int(np.ceil(numWind)),w))

# out[0]=windows for X;1-> Y; 2->Z axis
out=windowing(X,Y,Z,outX,outY,outZ,w,s,numRows)

result=statistics(out,numWind)

meanX=result[0]
meanY=result[2]
meanZ=result[4]
stdX=result[1]
stdY=result[3]
stdZ=result[5]

y=0
cc=0 #for calculating win range start index
rs=[]
while y<int(np.ceil(numWind)):
  rD=[str(cc)+'-'+str(cc+w-1)]
  rD.append(meanX[y])
  rD.append(stdX[y])
  rD.append(meanY[y])
  rD.append(stdY[y])
  rD.append(meanZ[y])
  rD.append(stdZ[y])
  rs.append(rD)
  y+=1
  cc+=s


df = pd.DataFrame(rs, columns = ['Win.Range','Mean X','Std X','Mean Y','Std Y','Mean Z','Std Z'])

print("Windowing(Time Domain)")
print( )
print(df)
# print(out[0])


Windowing(Time Domain)

   Win.Range    Mean X     Std X    Mean Y     Std Y    Mean Z     Std Z
0      0-255  0.008305  0.305307  0.117446  0.277708  9.762245  0.247507
1     16-271  0.046519  0.143848  0.100799  0.125455  9.755062  0.203499
2     32-287  0.033276  0.012564  0.110264  0.014560  9.776535  0.017168
3     48-303  0.033014  0.012613  0.110451  0.014588  9.777041  0.016584
4     64-319  0.033033  0.013159  0.110395  0.015099  9.777434  0.016769
5     80-335  0.033575  0.013192  0.110339  0.015443  9.777621  0.016773
6     96-351  0.033538  0.013124  0.110226  0.015380  9.777377  0.016630
7    112-367  0.028076  0.218987  0.109759  0.224702  9.566033  1.540301
8    128-383  0.025888  0.219053  0.102913  0.226230  8.955025  2.777723
9    144-399  0.023980  0.219123  0.096329  0.227569  8.343961  3.508864
10   160-415  0.021305  0.219112  0.089446  0.228679  7.732711  4.020069
11   176-431  0.019060  0.219119  0.082544  0.229573  7.121684  4.389044
12   192-447  0.017414  0.2

In [4]:
# Data preprocessing(Frequency domain)(for acceleration in X axis, window size=256)
'''out[0] contains list of acc in X for each window '''

# import sympy
from sympy import fft
from sympy import ifft

# to calculate fft for each window of acc in X
def FFT(out0,dec):
  c=0
  for r in out0:
    if c==0:
      transform=[fft(r,dec)]
      c+=1
    else:
      transform.append(fft(r,dec))  
  return transform

# sequence=out[0]
decimal_point = 4

# to calculate inverse fft for each window of fft of acc in X
def IFFT(trans,dec):
  c=0
  for r in trans:
    if c==0:
      itransform=[ifft(r,dec)]
      c+=1
    else:
      itransform.append(ifft(r,dec))  
  return itransform

# sequence=out[0]
decimal_point = 4

# fft
transform = FFT(out[0], decimal_point )
print ("FFT : ", transform)
print(len(transform))
print(len(transform[2]))
print( )

# inverse fft, seq=transform(which is fft seq)
itransform = IFFT(transform, decimal_point )
print ("IFFT : ", itransform)
print(len(itransform))
print(len(itransform[2]))

print( )
x=itransform
x=np.array(x)
if (x==out[0]).all():
  print("Inverse FFT achieved")
else:
  print("Inverse FFT failed")  


FFT :  [[2.12610000000000, -6.34105470990492 - 1.83818902211725*I, -5.54132441491734 - 3.30093464752023*I, -4.99230680271632 - 4.88685529195068*I, -4.17080519322922 - 6.03216664458855*I, -2.14985594922878 - 7.31108293517325*I, -0.288437403425282 - 8.21540891583375*I, 1.6841201498677 - 8.49043311495596*I, 3.93804380534816 - 8.50702654027086*I, 6.40651243958487 - 7.50898276067938*I, 8.53178259336682 - 6.17536713793713*I, 9.71176698186391 - 3.81626675999461*I, 10.9788770150977 - 1.15850434669781*I, 10.6224099126978 + 1.71748209789126*I, 9.69858816002098 + 4.53074899731345*I, 8.00754270220026 + 6.24439710855827*I, 5.67173221536805 + 7.82045106370038*I, 2.8364187572022 + 8.23438138032129*I, 0.479339329749547 + 8.0392708284993*I, -1.98658873349587 + 6.63311369953234*I, -3.18073826149359 + 4.99569089964887*I, -3.98610355875776 + 2.83462433342392*I, -4.03351163692901 + 0.389743287120458*I, -2.90681122622891 - 1.20988048118862*I, -1.04094158684194 - 2.82500556093095*I, 1.11500677570917 - 3.5475

In [22]:
# calculating root mean square for fft coefficients
# for each window for acceleration in X axis

import numpy as np
import scipy.signal
from cmath import sqrt

def rms(signal):
    """Computes root mean square of the signal.
    Square root of the arithmetic mean (average) of the squares of the original values.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which root mean square is computed
    Returns
    -------
    float
        Root mean square
    """
    return np.sqrt(np.sum(np.array(signal) ** 2) / len(signal))  
    # ii=np.array(np.sum(np.array(signal) ** 2) / len(signal))
    # jj=ii.astype(float)
    # return np.sqrt(ii)

# print(transform)
# RMS=[]
for winSignal in transform:
  x=rms(winSignal)
  print(x)
#   RMS.append(x) 

# print("RMS for each window's acc X:")
# for i in RMS:
#   print(i)     

TypeError: ignored

In [ ]:
# entropy
def spectral_entropy(signal, fs):
    """Computes the spectral entropy of the signal based on Fourier transform.
    Feature computational cost: 1
    Parameters
    ----------
    signal : nd-array
        Input from which spectral entropy is computed
    fs : int
        Sampling frequency
    Returns
    -------
    float
        The normalized spectral entropy value
    """
    # Removing DC component
    sig = signal - np.mean(signal)

    # f, fmag = calc_fft(sig, fs)

    power =  fmag** 2

    if power.sum() == 0:
        return 0.0

    prob = np.divide(power, power.sum())

    prob = prob[prob != 0]

    # If probability all in one value, there is no entropy
    if prob.size == 1:
        return 0.0

    return -np.multiply(prob, np.log2(prob)).sum() / np.log2(prob.size)